# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [60]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string
from collections import Counter 

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## load packages 
import pandas as pd
import re
import numpy as np

## nltk imports
#!pip install nltk # can install on terminal or by uncommenting this line
#import nltk; nltk.download('punkt'); nltk.download('stopwords')
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

## sklearn imports
from sklearn.feature_extraction.text import CountVectorizer

## lda
#!pip install gensim # can install by uncommenting this line
from gensim import corpora
import gensim

## visualizing LDA--likely need to install
#!pip install pyLDAvis # can install by uncommenting this line
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
pyLDAvis.enable_notebook()

## print mult things
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## random
import random
import string; punctstr = string.punctuation # english punctuation marks

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/katinachristensen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

## 0.1 Load and clean text data

In [3]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [4]:
## your code to subset to one press release and take the string 
string = ' '.join(doj['contents'].astype(str)) 

# Filter the DataFrame to get the row corresponding to the press release with id "17-1204"
subset_data = doj[doj['id'] == "17-1204"]

# Convert the contents column of the subset to a single string
pharma = subset_data['contents'].iloc[0] 

pharma 

'The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspi

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [5]:
## your code here to restrict to alpha
pharma_cleaned = ''.join(char for char in pharma if char.isalpha() or char.isspace())

In [6]:
## your code here for part of speech tagging and part C 

# Tokenize the cleaned press release text into words
tokens = word_tokenize(pharma_cleaned)

# Perform part-of-speech tagging
pos_tags = nltk.pos_tag(tokens)

# Part C 
# Filter adjectives from pos_tags
adjectives = [word for word, pos in pos_tags if pos.startswith('JJ')]

# Count the occurrences of each adjective
adjectives_count = Counter(adjectives)

# Sort adjectives by frequency
sorted_adjectives = sorted(adjectives_count.items(), key=lambda x: x[1], reverse=True)

# Convert the sorted adjectives to a DataFrame
df_top_adjectives = pd.DataFrame(sorted_adjectives, columns=['Adjective', 'Count'])

# Select the top 5 most frequent adjectives
top_5_adjectives = df_top_adjectives.head(5)

top_5_adjectives 

,Adjective,Count
0,former,8
1,opioid,5
2,nationwide,4
3,other,3
4,addictive,3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [7]:
## your code here for part A
# Process the original pharmaceutical press release text
doc = nlp(pharma)

# Extract named entities
named_entities = [(ent.text, ent.label_) for ent in doc.ents]

# Print the named entities
for entity, label in named_entities:
    print(f"{entity}: {label}")

Insys Therapeutics Inc.: ORG
today: DATE
Fentanyl: PERSON
More than 20,000: CARDINAL
Americans: NORP
last year: DATE
millions: CARDINAL
Jeff Sessions: PERSON
This Justice Department: ORG
Trump: PERSON
American: NORP
”John N. Kapoor: PERSON
74: DATE
Phoenix: GPE
Ariz.: GPE
the Board of Directors: ORG
Insys: ORG
this morning: TIME
Arizona: GPE
RICO: LAW
Kapoor: PERSON
Executive: ORG
Board: ORG
Insys: ORG
Phoenix: GPE
today: DATE
U.S.: GPE
District Court: ORG
Boston: GPE
a later date: DATE
today: DATE
Boston: GPE
Insys: ORG
December 2016.The: DATE
Kapoor: GPE
Michael L. Babich: PERSON
40: DATE
Scottsdale: GPE
Ariz.: GPE
Alec Burlakoff: PERSON
42: DATE
Charlotte: GPE
N.C.: GPE
Richard M. Simon: PERSON
46: DATE
Seal Beach: GPE
Calif.: GPE
Sunrise Lee: PERSON
36: DATE
Bryant City: GPE
Mich.: GPE
Joseph A. Rowan: PERSON
43: DATE
Panama City: GPE
Fla.: GPE
Managed Markets: ORG
Michael J. Gurry: PERSON
53: DATE
Scottsdale: GPE
Ariz.: GPE
Subsys: ORG
Kapoor: GPE
six: CARDINAL
Kapoor: PERSON
Unit

In [8]:
## your code here for part B
# Extract unique named entities with the tag "LAW"
law_entities = set([(ent.text, ent.label_) for ent in doc.ents if ent.label_ == "LAW"])

# Print the unique named entities with the tag "LAW"
for entity, label in law_entities:
    print(f"{entity}: {label}")

the Controlled Substances Act: LAW
RICO: LAW


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

- RICO (Racketeer Influenced and Corrupt Organizations Act) is a U.S. federal law targeting organized crime (like a mafia case) but can also apply to cases involving illegal activities by corporations or individuals connected to a criminal enterprise, such as pharmaceutical kickbacks, becasue RICO includes various forms of racketeering and unlawful behavior. 

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [9]:
## your code here
# Extract named entities with the label "DATE" and containing the words "year" or "years"
sentence_lengths = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ == "DATE" and re.search(r'\b(?:year|years)\b', ent.text, flags=re.IGNORECASE)]

# Print the extracted named entities
for entity, label in sentence_lengths:
    print(f"{entity}: {label}")

last year: DATE
20 years: DATE
three years: DATE
five years: DATE
three years: DATE


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [10]:
## your code here --- NEED TO FIX THIS

# Convert the pharma press release into sentences
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', pharma)

# Find and print the sentences containing the relevant named entities
sentences_containing_years = []
for sentence in sentences:
    if any(re.search(r'\b(?:year|years)\b', sentence, flags=re.IGNORECASE) for entity in sentence_lengths):
        sentences_containing_years.append(sentence)

print("Sentences containing year lengths:")
for sentence in sentences_containing_years:
    print(sentence)


Sentences containing year lengths:
"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.
Neves, Special Agent in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.
 The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.


## Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum).
- If convicted after this indictment, the CEO may face a maximum sentence of 20 years in prison.

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [11]:
## your code here for subsetting
doj_subset = doj[doj['topics_clean'].isin(['Civil Rights', 'Hate Crimes', 'Project Safe Childhood'])]
print("There are " + str(len(doj_subset)) + " rows in doj_subset.") 
doj_subset 

There are 717 rows in doj_subset.


,id,title,contents,date,topics_clean,components_clean
77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle"
155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.",2015-12-11T00:00:00-05:00,Project Safe Childh

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [12]:
## your code here to define function
def analyze_press_release(string: str):
    "''Press Release Sentiment Analyzer"''
    # Remove named entities
    named_entities = nlp(string)
    for entity in named_entities.ents:
        if entity.text.isalpha():
            string = re.sub(re.escape(entity. text), "", string)
    
    # Get sentiment scores
    sent_obj = SentimentIntensityAnalyzer()
    sentiment = sent_obj. polarity_scores(string)
    return sentiment

In [16]:
## your code here executing the function

# Apply the function to each press release in the doj_subset DataFrame
sentiment_scores = doj_subset['contents'].apply(lambda x: analyze_press_release(x))

sentiment_scores 

77        {'neg': 0.172, 'neu': 0.758, 'pos': 0.07, 'compound': -0.9893}
155      {'neg': 0.121, 'neu': 0.769, 'pos': 0.111, 'compound': -0.7003}
157       {'neg': 0.091, 'neu': 0.803, 'pos': 0.105, 'compound': 0.6124}
162      {'neg': 0.118, 'neu': 0.772, 'pos': 0.111, 'compound': -0.5385}
168      {'neg': 0.149, 'neu': 0.792, 'pos': 0.059, 'compound': -0.9786}
                                      ...                               
13002     {'neg': 0.14, 'neu': 0.794, 'pos': 0.066, 'compound': -0.9645}
13032      {'neg': 0.078, 'neu': 0.798, 'pos': 0.123, 'compound': 0.946}
13034    {'neg': 0.133, 'neu': 0.729, 'pos': 0.138, 'compound': -0.0516}
13068     {'neg': 0.13, 'neu': 0.735, 'pos': 0.135, 'compound': -0.2054}
13081    {'neg': 0.133, 'neu': 0.832, 'pos': 0.035, 'compound': -0.9898}
Name: contents, Length: 717, dtype: object

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [18]:
## your code here

# Convert the resulting Series of dictionaries to a DataFrame
sentiment_df = pd.DataFrame(sentiment_scores.tolist(),index = sentiment_scores.index)
# Concatenate the sentiment DataFrame with the original doj_subset DataFrame
doj_subset_wscore = pd.concat([doj_subset, sentiment_df], axis=1)

# Sort the DataFrame by the 'negative' score in descending order
doj_subset_wscore = doj_subset_wscore.sort_values(by='neg', ascending=False) 

# Print the top two most negative 
top_2_neg = doj_subset_wscore.head(2)
top_2_neg[["id", "contents", "neg"]] 

,id,contents,neg
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",0.276
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.268


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [19]:
## agg and find the mean compound score by topic
mean_compound_scores = doj_subset_wscore.groupby('topics_clean').agg({'compound': 'mean'})

mean_compound_scores 

,compound
topics_clean,
Civil Rights,0.151243
Hate Crimes,-0.895692
Project Safe Childhood,-0.244107


## E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)
- One reason why the topics of Civil Rights may recive more positive score compare to Hate Crime and Project Safe Childhood is because Civil Rights may be spoken about with more positive language then the other two categories, which could be discussed with more neutral or pessimistic language.  

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [20]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [31]:
# Custom stopwords to add
custom_stopwords = ["custom_stopword1", "custom_stopword2"]

def preprocess_text(raw_text):
    # Convert text to lowercase
    raw_text = raw_text.lower()
    
    # Tokenize text into words
    words = nltk.word_tokenize(raw_text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english') + custom_stopwords)
    filtered_words = [word for word in words if word not in stop_words]
    
    # Remove punctuation and digits, and retain only alpha words
    filtered_words = [word for word in filtered_words if word.isalpha()]
    
    # Retain words 4 characters or longer
    filtered_words = [word for word in filtered_words if len(word) >= 4]
    
    # Stemming using SnowballStemmer
    stemmer = SnowballStemmer("english")
    stemmed_words = [stemmer.stem(word) for word in filtered_words]
    
    # Join preprocessed words into a single string
    preprocessed_text = ' '.join(stemmed_words)
    
    return preprocessed_text

In [32]:
## your code executing the function
# Apply the preprocess_text function to each row in the DataFrame and create a new column 'processed_text'
doj_subset_wscore['processed_text'] = doj_subset_wscore['contents'].apply(preprocess_text)

In [33]:
## your code showing the examples
examples1 = doj_subset_wscore[doj_subset_wscore['id'].isin(['16-718', '16-217'])][['id', 'contents', 'processed_text']]
examples1 

,id,contents,processed_text
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti case investig jackson divis cooper mississippi depart correct prosecut assist attorney robert coleman northern district mississippi trial attorney dana mulhaus civil right divis crimin section marsher indict
6727,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [39]:
def create_dtm(list_of_strings, metadata):
    # Initialize CountVectorizer
    vectorizer = CountVectorizer(lowercase=True)
    
    # Transform the list of strings into a document-term matrix
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    
    # Convert the sparse matrix into a dense DataFrame with feature names
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names_out())
    
    # Drop any columns from dtm_dense_named that are already in metadata
    dtm_dense_named = dtm_dense_named.drop(columns=metadata.columns, errors='ignore')
    
    # Concatenate metadata with the dense DataFrame
    dtm_dense_named_withid = pd.concat([metadata.reset_index(drop=True), dtm_dense_named], axis=1)
    
    return dtm_dense_named_withid

# Replace 'doj_subset_wscore' and 'processed_text' with your actual data
list_of_strings = doj_subset_wscore['processed_text']
metadata = doj_subset_wscore[['id', 'compound', 'topics_clean']]

# Create DTM
dtm = create_dtm(list_of_strings, metadata)
print(dtm)


          id  compound  topics_clean  aaron  abandon  abbat  abbi  abbott  \
0     16-718   -0.9968  Civil Rights      0        0      0     0       0   
1     14-248   -0.9943   Hate Crimes      0        0      0     0       0   
2     13-312   -0.9980   Hate Crimes      0        0      0     0       0   
3    15-1348   -0.9963  Civil Rights      0        0      0     0       0   
4     11-626   -0.9985   Hate Crimes      0        0      0     0       0   
..       ...       ...           ...    ...      ...    ...   ...     ...   
712   17-132    0.9905  Civil Rights      0        0      0     0       0   
713  15-1559    0.9460  Civil Rights      0        0      0     0       0   
714   16-294    0.9442  Civil Rights      0        0      0     0       0   
715   15-667    0.9118  Civil Rights      0        0      0     0       0   
716   17-271    0.9949  Civil Rights      0        0      0     0       0   

     abdomen  abduct  ...  zamora  zane  zealand  zealous  zeeman  zero  \


In [40]:
# your code here
list = ['id', 'compound', 'topics_clean']
data_for_dtm = doj_subset_wscore['processed_text']
dtm = create_dtm(data_for_dtm, metadata = doj_subset_wscore[list])
dtm 

,id,compound,topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,abduct,...,zamora,zane,zealand,zealous,zeeman,zero,zobel,zone,zunggeemog,zwengel
0,16-718,-0.9968,Civil Rights,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14-248,-0.9943,Hate Crimes,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,13-312,-0.9980,Hate Crimes,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15-1348,-0.9963,Civil Rights,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11-626,-0.9985,Hate Crimes,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,17-132,0.9905,Civil Rights,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,15-1559,0.9460,Civil Rights,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,16-294,0.9442,Civil Rights,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,15-667,0.9118,Civil Rights,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# Step 1: Filter press releases with compound sentiment in the top 5% and bottom 5%
top_5_percentile = doj_subset_wscore['compound'].quantile(0.95)
bottom_5_percentile = doj_subset_wscore['compound'].quantile(0.05)

positive_sentiment = doj_subset_wscore[doj_subset_wscore['compound'] >= top_5_percentile]
negative_sentiment = doj_subset_wscore[doj_subset_wscore['compound'] <= bottom_5_percentile]

# Step 2: Tokenize the text and create a Document-Term Matrix (DTM)
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(lowercase=True)
dtm_positive = vectorizer.fit_transform(positive_sentiment['processed_text'])
dtm_negative = vectorizer.fit_transform(negative_sentiment['processed_text'])

# Step 3: Calculate word frequencies
positive_word_freq = dtm_positive.sum(axis=0)
negative_word_freq = dtm_negative.sum(axis=0)

# Step 4: Print the top 10 words for each group
word_list = vectorizer.get_feature_names_out()

print("Top 10 words for press releases with positive sentiment:")
for idx in positive_word_freq.argsort()[0, -10:]:
    print(word_list[idx])

print("\nTop 10 words for press releases with negative sentiment:")
for idx in negative_word_freq.argsort()[0, -10:]:
    print(word_list[idx])


Top 10 words for press releases with positive sentiment:
[['done' 'depart' 'magnitud' 'day' 'card' 'aspect' 'identifi' 'program'
  'aggress' 'credit']]

Top 10 words for press releases with negative sentiment:
[['justic' 'divis' 'civil' 'victim' 'depart' 'crime' 'assault' 'right'
  'attorney' 'offic']]


In [47]:
# Step 1: Get unique topics
unique_topics = doj_subset_wscore['topics_clean'].unique()

# Step 2: Iterate over each topic and print the top 10 words
for topic in unique_topics:
    # Filter dataset for the current topic
    topic_data = doj_subset_wscore[doj_subset_wscore['topics_clean'] == topic]
    
    # Tokenize the text and create a Document-Term Matrix (DTM)
    vectorizer = CountVectorizer(lowercase=True)
    dtm_topic = vectorizer.fit_transform(topic_data['processed_text'])
    
    # Summing each column
    word_freq = dtm_topic.sum(axis=0)
    
    # Convert to DataFrame
    word_freq_df = pd.DataFrame(word_freq, columns=vectorizer.get_feature_names_out())
    
    # Sorting from most frequent to least frequent
    top_terms = word_freq_df.sum().sort_values(ascending=False)
    
    # Print the top 10 words for the current topic
    print(f"\nTop 10 words for press releases in topic: {topic}")
    print(top_terms.head(10))



Top 10 words for press releases in topic: Civil Rights
depart      1565
right       1335
civil       1062
justic      1003
attorney     954
offic        943
divis        802
district     740
hous         620
assist       550
dtype: int64

Top 10 words for press releases in topic: Hate Crimes
attorney    1088
right       1023
civil        789
depart       660
divis        644
victim       590
district     569
justic       557
crime        533
assist       508
dtype: int64

Top 10 words for press releases in topic: Project Safe Childhood
child        1018
attorney      878
exploit       698
district      590
sexual        570
case          485
safe          476
project       472
childhood     472
depart        448
dtype: int64


## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [64]:
from gensim import corpora, models

# Step 1: Convert preprocessed words to a list of lists
preprocessed_words = doj_subset_wscore['processed_text'].apply(lambda x: x.split())
preprocessed_words_list = preprocessed_words.tolist()

# Step 2: Create a dictionary
text_dict = corpora.Dictionary(preprocessed_words_list)

# Step 3: Filter out very rare and very common words
text_dict.filter_extremes(no_below=5, no_above=0.95)

# Step 4: Convert words to bag of words format
corpus = [text_dict.doc2bow(text) for text in preprocessed_words_list]

# Step 5: Estimate the topic model with 3 topics
lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word=text_dict, passes=20)

# Step 6: Print the top 15 words in each topic
topics = lda_model.show_topics(num_topics=3, num_words=15, formatted=False)

for i, topic in topics:
    print(f"\nTopic {i+1}:")
    for word, prob in topic:
        print(f"{word}")



Topic 1:
child
exploit
sexual
district
case
safe
project
childhood
pornographi
children
crimin
investig
prosecut
victim
sentenc

Topic 2:
right
civil
offic
hous
discrimin
district
disabl
enforc
said
agreement
state
violat
court
investig
alleg

Topic 3:
right
civil
district
victim
charg
sentenc
prosecut
investig
case
offic
defend
guilti
crime
feder
said


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [67]:
# Step 1: Extract document-level topic probabilities
document_topic_probs = [lda_model.get_document_topics(doc, minimum_probability=0) for doc in corpus]

# Step 2: Assert statement to ensure the length of the list is equal to the number of rows in the dataframe
assert len(document_topic_probs) == len(doj_subset_wscore), "Length of document-level topic probabilities does not match the number of rows in the dataframe"

# Step 3: Add topic probabilities to the dataframe as columns
topic_columns = [f"topic_{i+1}" for i in range(lda_model.num_topics)]
for i, topic_probs in enumerate(document_topic_probs):
    for topic, prob in topic_probs:
        doj_subset_wscore.loc[i, f"topic_{topic+1}_prob"] = prob

# Step 4: Create a column 'top_topic' reflecting each document's highest-probability topic
doj_subset_wscore['top_topic'] = doj_subset_wscore[[f"topic_{i+1}_prob" for i in range(lda_model.num_topics)]].idxmax(axis=1).apply(lambda x: int(x.split("_")[1]))

# Step 5: Display the dataframe with added columns
print(doj_subset_wscore.head())


AssertionError: Length of document-level topic probabilities does not match the number of rows in the dataframe

In [50]:
## your code here to add those topic probabilities to the dataframe
# Convert the list of document-level topic probabilities to a DataFrame
topic_probabilities_df = pd.DataFrame(document_topics, columns=[f"topic_{i+1}" for i in range(n_topics)])

# Add the topic probabilities to the doj_subset_wscore dataframe
doj_subset_wscores_with_topics = pd.concat([doj_subset_wscore.reset_index(drop=True), topic_probabilities_df], axis=1)

# Create a column 'top_topic' that reflects each document's highest-probability topic
doj_subset_wscores_with_topics['top_topic'] = doj_subset_wscores_with_topics[[f"topic_{i+1}" for i in range(n_topics)]].idxmax(axis=1)
doj_subset_wscores_with_topics['top_topic'] = doj_subset_wscores_with_topics['top_topic'].str.replace('topic_', '').astype(int)

# Display the modified dataframe
doj_subset_wscores_with_topics 

,id,title,contents,date,topics_clean,components_clean,neg,neu,pos,compound,processed_text,topic_1,topic_2,topic_3,top_topic
0,16-718,Three Mississippi Correctional Officers Indicted for Inmate Assault and Cover-Up,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",2016-06-21T00:00:00-04:00,Civil Rights,"Civil Rights Division; Civil Rights - Criminal Section; USAO - Mississippi, Northern",0.276,0.694,0.030,-0.9968,indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti case investig jackson divis cooper mississippi depart correct prosecut assist attorney robert coleman northern district mississippi trial attorney dana mulhaus civil right divis crimin section marsher indict,0.554206,0.002584,0.443210,1
1,14-248,Albuquerque Man Charged with Federal Hate Crime Related to Anti-Semitic Threats Against Businesswoman,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint 

In [51]:
## your code here to summarize the topic proportions for each of the topics_clean 
# Calculate the percentage breakdown of top topics for each manual label
topic_breakdown = pd.crosstab(doj_subset_wscores_with_topics['topics_clean'], doj_subset_wscores_with_topics['top_topic'], normalize='index') * 100

# Display the percentage breakdown
print("Percentage breakdown of top topics for each manual label:")
topic_breakdown 

Percentage breakdown of top topics for each manual label:


top_topic,1,2,3
topics_clean,,,
Civil Rights,20.000000,66.557377,13.442623
Hate Crimes,0.406504,0.406504,99.186992
Project Safe Childhood,100.000000,0.000000,0.000000


## D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)
- The mapping between manual topics and estimated topics may vary due to the complexity of the topics. Topics such as "Hate Crime" or "Project Safe Childhood," which represent more specific and distinct themes, may map more cleanly to estimated topics because they are likely to exhibit stronger patterns in the underlying data; alternately, broader topics like "Civil Rights" may encompass a wider range of issues and perspectives, leading to more diffuse mappings with estimated topics as they might intersect with multiple themes identified by the model. 

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [52]:
## your code here 
import pandas as pd

# Assuming 'doj_subset_wscore' is the DataFrame containing the processed text in the 'processed_text' column

# Function to create bigrams from a list of words
def create_bigrams(text):
    words = text.split()
    bigrams = [f"{words[i]}_{words[i+1]}" for i in range(len(words)-1)]
    return " ".join(bigrams)

# Apply the function to create bigrams for each row in the 'processed_text' column
doj_subset_wscore['processed_text_bigrams'] = doj_subset_wscore['processed_text'].apply(create_bigrams)

# Display the appropriate data from B 
doj_subset_wscore[doj_subset_wscore['id'] == '16-217'][['id', 'processed_text', 'processed_text_bigrams']]

,id,processed_text,processed_text_bigrams
6727,16-217,justic depart reach comprehens settlement agreement citi miami miami polic depart resolv justic depart investig shoot offic announc princip deputi assist attorney general vanita gupta head justic depart civil right divis attorney wifredo ferrer southern district florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem justic depart investig shoot offic conduct violent crime control enforc investig find issu juli identifi pattern practic excess forc shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim shoot effect quick investig shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi assist attorney general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc today agreement result joint effort depart justic citi miami ensur miami polic depart continu effort make communiti safe protect sacr constitut right citizen said attorney ferrer oversight communic agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss settlement agreement build upon import reform implement citi sinc justic depart issu find includ investig conduct attorney staff civil right divis special litig section civil divis attorney offic southern district florida,justic_depart depart_reach reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_depart depart_resolv resolv_justic justic_depart depart_investig investig_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_assist assist_attorney attorney_general general_vanita vanita_gupta gupta_head head_justic justic_depart depart_civil civil_right right_divis divis_attorney attorney_wifredo wifredo_ferrer ferrer_southern southern_district district_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_justic justic_depart depart_investig investig_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_investig investig_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_shoot shoot_effect effect_quick quick_investig investig_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_assist assist_attorney attorney_general general_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_today today_agreement agreement_result result_joint joint_effort effort_depart d

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [54]:
# your code here
def create_dtm_with_bigrams(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase=True, ngram_range=(1, 2))  # Include unigrams and bigrams
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(drop=True), dtm_dense_named], axis=1)
    return dtm_dense_named_withid

# Create the document-term matrix with bigrams
dtm_bigram = create_dtm_with_bigrams(doj_subset_wscore['processed_text_bigrams'], doj_subset_wscore[['id', 'topics_clean', 'compound']])

# Display the shape of each matrix
dtm_bigram 

# Print the dimensions of the dtm matrix
print("The dimensions of the dtm matrix: " + str(dtm.shape)) 

# Print the dimensions of the dtm_bigram matrix
print("The dimensions of the dtm_bigram matrix: " + str(dtm_bigram.shape)) 

,id,topics_clean,compound,aaron_ford,aaron_ford ford_memphi,aaron_latham,aaron_latham latham_munci,aaron_mcgrath,aaron_mcgrath mcgrath_websit,aaron_parrish,...,zone_varianc,zone_varianc varianc_would,zunggeemog_noel,zunggeemog_noel noel_womack,zunggeemog_prompt,zunggeemog_prompt prompt_forc,zunggeemog_write,zunggeemog_write write_report,zwengel_princeton,zwengel_princeton princeton_illinoi
0,16-718,Civil Rights,-0.9968,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14-248,Hate Crimes,-0.9943,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,13-312,Hate Crimes,-0.9980,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15-1348,Civil Rights,-0.9963,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11-626,Hate Crimes,-0.9985,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,17-132,Civil Rights,0.9905,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,15-1559,Civil Rights,0.9460,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,16-294,Civil Rights,0.9442,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,15-667,Civil Rights,0.9118,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The dimensions of the dtm matrix: (717, 6758)
The dimensions of the dtm_bigram matrix: (717, 177110)


## Comment on why the bigram matrix has more dimensions than the unigram matrix
- The bigram matrix has more dimensions than the unigram matrix because it considers not only individual words (unigrams) but also pairs of consecutive words: bigrams. Each bigram represents a unique combination of two words, effectively expanding the space of the matrix. In turn, the bigram matrix captures more detailed information about word sequences and dependencies, leading to a higher-dimensional representation compared to using unigrams alone. 

In [190]:
# Part E 

# Define a function to print the top bigrams for each topic
def print_top_bigrams_for_topic(topic):
    # Filter data for the specific topic
    topic_subset = dtm_bigram[dtm_bigram['topics_clean'] == topic]
    # Get the top bigrams for the topic
    top_bigrams = get_topwords(topic_subset.drop(['id', 'topics_clean', 'compound'], axis=1))
    # Print the top bigrams
    print(f"Top 10 bigrams for the '{topic}' topic:")
    print(top_bigrams)
    print()

# Print top bigrams for each topic
topics = dtm_bigram['topics_clean'].unique()
for topic in topics:
    print_top_bigrams_for_topic(topic)

Top 10 bigrams for the 'Civil Rights' topic:
civil_right                         948
justic_depart                       720
civil_right right_divis             679
right_divis                         679
assist_attorney                     392
attorney_general                    379
assist_attorney attorney_general    300
fair_hous                           227
depart_civil                        224
deputi_assist                       223
dtype: int64

Top 10 bigrams for the 'Hate Crimes' topic:
civil_right                         769
civil_right right_divis             523
right_divis                         523
assist_attorney                     421
hate_crime                          368
justic_depart                       275
plead_guilti                        275
attorney_general                    238
assist_attorney attorney_general    223
trial_attorney                      214
dtype: int64

Top 10 bigrams for the 'Project Safe Childhood' topic:
project_safe safe_childhood 

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [191]:
# your code here 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'doj_subset' is the DataFrame containing processed press releases

# Create TF-IDF vectors for the processed press releases
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(doj_subset_wscore['processed_text'])

# Calculate pairwise cosine similarity between the press releases
pairwise_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Find the top two pairs with the highest similarity
n_pairs = 2
top_pairs = []
for i in range(len(pairwise_similarity)):
    for j in range(i+1, len(pairwise_similarity)):
        similarity = pairwise_similarity[i][j]
        top_pairs.append((i, j, similarity))

# Sort the pairs by similarity in descending order
top_pairs.sort(key=lambda x: x[2], reverse=True)

# Print the top two pairs
for idx1, idx2, similarity in top_pairs[:n_pairs]:
    print("Pair " + str(n_pairs) + ":")
    print("Press Release 1:")
    print(doj_subset_wscore.iloc[idx1]['processed_text'])
    print("\nPress Release 2:")
    print(doj_subset_wscore.iloc[idx2]['processed_text'])
    print("Similarity:", similarity)
    print("----")

Pair 2:
Press Release 1:
church hill maryland resid sentenc today year prison follow lifetim term supervis releas entic minor engag sexual activ attempt transfer obscen materi minor announc act assist attorney general kenneth blanco justic depart crimin divis act attorney benjamin greenberg southern district florida robert moor plead guilti march district judg daniel hurley southern district florida moor employ secret divis assign white hous time arrest remain custodi sinc time moor sinc termin secret servic posit accord admiss made connect plea moor maintain profil social media applic provid platform exchang digit imag well voic text messag delawar state polic detect delawar child predat task forc creat profil site pose girl moor engag number onlin chat session mobil app period includ moor work number onlin chat moor undercov offic pose femal minor sexual natur sever occas moor sent pictur includ sexual explicit imag accord plea document arrest enforc discov moor communic minor florid